## Least Cost Pipeline Construction -- All NG
Sites are added in random order to an organic pipeline network.

In [13]:
#Import packages
import numpy as np
import pandas as pd
from skimage import graph
from osgeo import gdal
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
#Read the cost and cost distance stacks
arrCosts = np.load('../data/DuplinCostStack.npy')
arrStack = np.load('../data/DuplinStack.npy')

In [15]:
#Read in the pipeline raster
ds =  gdal.Open('../data/processed/duplin_pipe_diameter.tif')
#Get the raster projection
ds_prj = ds.GetProjection()
ds_WKT = 'PROJCS["North_America_Equidistant_Conic",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Equidistant_Conic"],PARAMETER["False_Easting",0],PARAMETER["False_Northing",0],PARAMETER["central_meridian",-96],PARAMETER["Standard_Parallel_1",20],PARAMETER["Standard_Parallel_2",60],PARAMETER["latitude_of_origin",40],UNIT["Meter",1]]'
#Get raster attributes (used in MCP analysis)
llx, x_size, x_angle, lly, y_angle, y_size = ds.GetGeoTransform()
#Extract Band1 as the cost array; divide by 100 to scale as a percentage
arrPipesAll = np.array(ds.GetRasterBand(1).ReadAsArray())
#Convert all pipe pixels to 1, others to -1
arrPipesAll[arrPipesAll >= 0] = 1
arrPipesAll[arrPipesAll < 0] = 0
#Convert to a masked array (ignoring all values < 0, which are NoData)
arrPipes = np.ma.masked_array(arrPipesAll, mask=arrPipesAll < 0)

In [16]:
#Read in biogas sources (example: Duplin Co)
dfBG =  pd.read_excel('../data/DuplinCountySwineFarmEconomics.xlsx',
                         sheet_name='Duplin County Swine Farm Master').iloc[:,[11,12,17,19,22]]
#Sort values in ascending order by cost to pipe
dfBG.sort_values(by='TEST ($/mi-MMBtu @15y)',ascending=True,inplace=True)
#Set the master index as the value of indices
master_index = dfBG.index.tolist()

## Functions

In [17]:
def getPipeCoord(array_item,array_mask,verbose=False):
    '''Returns the layer and row/col index of the pixel in a stack of layers containing the 
       overall minimum value found among cells within the supplied mask array.
    
    Arguments:
     array_stack (3d array): Stack of cost distance layers from each biogas source
     array_mask (2d array): Layer of pixels from which min values should be returned
     verbose (Boolean): Set to true to display processing messages
    
    Returns (tuple):
     [1] (integer) Index of the layer in the array_stack containing the minimum values
     [2] (tuple) Row/column coordinates of the pixel containing the minimum value.
     [3] (float) The vost value at the location
    
    '''
    #Reduce the stack of cost distance layers to a layer of the minimum value at each location
    #arrMin = np.amin(array_stack,axis=0)
    #Mask the values so only pipeline pixels are kept
    arrMin_masked = np.ma.masked_array(array_item, mask=array_mask != 1)
    #Find the min value of the pipeline pixels
    minValue = np.amin(arrMin_masked)
    #Determine the row and column where the min occurs
    rMin,cMin = np.where(arrMin_masked == minValue)
    #Extract values from value arrays
    rowMin = rMin[0];  colMin = cMin[0]
    if verbose: print("Link to LCP row/col ({},{} (Layer:{}, Cost:{:.2f}))".format(rowMin,colMin,layerIdx,minValue))
    return((rowMin,colMin),minValue)

In [18]:
def getPipeValue(array_stack,array_mask,verbose=False):
    '''Returns the layer and row/col index of the pixel in a stack of layers containing the 
       overall minimum value found among cells within the supplied mask array.
    
    Arguments:
     array_stack (3d array): Stack of cost distance layers from each biogas source
     array_mask (2d array): Layer of pixels from which min values should be returned
     verbose (Boolean): Set to true to display processing messages
    
    Returns (tuple):
     [1] (integer) Index of the layer in the array_stack containing the minimum values
     [2] (tuple) Row/column coordinates of the pixel containing the minimum value.
     [3] (float) The vost value at the location
    
    '''
    #Reduce the stack of cost distance layers to a layer of the minimum value at each location
    arrMin = np.amin(array_stack,axis=0)
    #Mask the values so only pipeline pixels are kept
    arrMin_masked = np.ma.masked_array(arrMin, mask=array_mask != 1)
    #Find the min value of the pipeline pixels
    minValue = np.amin(arrMin_masked)
    #Determine the row and column where the min occurs
    rMin,cMin = np.where(arrMin_masked == minValue)
    #Extract values from value arrays
    rowMin = rMin[0];  colMin = cMin[0]
    #Find which layer has the min value at this location in the stack of layers
    layers = np.where(array_stack[:,rowMin,colMin] == minValue)[0]
    #if len(layers) > 1: layerIdx = layers[0]
    #else: layerIdx = [layers]
    layerIdx = layers[0]
    if verbose: print("Link to LCP row/col ({},{} (Layer:{}, Cost:{:.2f}))".format(rowMin,colMin,layerIdx,minValue))
    return(layerIdx,(rowMin,colMin),minValue)

In [19]:
def getSourceCoordinates(array_stack,layer_id,verbose=False):
    '''Returns the row and column index for a biogas source in the provided layer
    
    Args: 
     array_stack (3d array): Stack of cost distance layers from each biogas source
     layer_id (integer): Index value of the layer containing the biogas source
     verbose (Boolean): Set to true to display processing messages
    
    Returns (tuple): 
     The Row/column coordinates of the pixel containing the minimum 
     value in the layer, i.e., the biogas source.
    
    '''
    #Get the layer from the stack corresponding to the source
    arrSource = array_stack[layer_id]
    #Find the minimum of that layer
    minValue = np.amin(arrSource)
    #Get the row and columns corresponding to that value
    rFarm,cFarm = np.where(array_stack[layer_id,:,:]==minValue)
    #Convert value arrays to just values
    theRow = rFarm[0]; theCol = cFarm[0]
    if verbose: print("Source occurs at ({},{})".format(theRow,theCol))
    #Return coords as a tuple
    return(theRow,theCol)

In [20]:
def getLCP(array_costs,layer_id,source_coords,pipe_coords,x_size=500,y_size=500):
    '''Returns a least cost layer between biogas source and pipeline coordinates
       for a given layer using the cost surface for a specific biogase source among
       the stack of biogas cost surface layers. 
       
    Args:
     array_costs (3d array): Stack of cost rasters (not cost distance) for each source
     layer_id (integer): Index value of the layer containing the biogas source
     source_coords (tuple): Row/column of the pixel containing the biogas source (start of LCP)
     pipe_coords (tuple): Row/columns of the pipeline pixel to connect (end of LCP)
     x_size (integer): Pixel width, in units of the cost layers [default=500(m)]
     y_size (integer): Pixel height, in units of the cost layers [default=500(m)]
     
    Returns (2d array):
     A 2D array of 1s and 0s where 1s represent the least cost pathway between biogas
     source and the pipeline coordinate provided. 
    '''
    #Extract the cost surface for that farm
    arrCost = array_costs[layer_id,:,:]
    #Create the MCP graph object from the farm location
    lc_graph = graph.MCP_Geometric(arrCost, sampling=(x_size,y_size))
    #Compute the cost distance and traceback arrays
    cd_array,tb_array = lc_graph.find_costs(starts=([source_coords]))
    #Get the row/col indices of pixels in the LCP 
    lcp_indices = lc_graph.traceback(pipe_coords)
    #Convert collection of indices to a layer
    arr_LCP = np.zeros(arrCost.shape) #Create a layer of all NaNs
    for r,c in lcp_indices:                 #Loop through coordinates in traceback
        arr_LCP[r,c] = 1                     # ..set values to 1
    #Return the lcp array
    return(arr_LCP,lcp_indices)

In [21]:
def coordsToline(coord_list,ll_x,ll_y,x_size,y_size):
    '''Convert a list of row/col coordinates to a line feature.
    
    Args: 
        coordList (list): collection of array coordinates
        ll_x (float): geographic x coordinate of lower left corner of array
        ll_y (float): geographic y coordinate of lower left corner of array
        x_size (float): pixel size in the x direction
        y_size (float): pixel size in the x direction
        
    Returns:
        Shapely linestring object
    
    '''
    #Import library
    from shapely.geometry import LineString
    #Initialize the list of geographic coordinates
    outCoords = []
    #Iterate through coordinates, georeference, and add to the list
    for row,col in coord_list:
        #Convert row and column indicies to geographic coordinates
        x = x_size * col + llx + (x_size/2)
        y = y_size * row + lly + (y_size/2)
        #Append to the list
        outCoords.append((x,y))
        
    #Construct a shapely linestring
    if len(outCoords) < 2:
        line = ((outCoords[0],outCoords[0]))
    else:
        line = LineString(outCoords)

    
    #Return the line
    return line

## Main code
1. Sort the biogas sources in random order 
2. Sequentially connect them in that order


In [22]:
#Generate a randomly sorted list of IDs
stackList = list(range(arrStack.shape[0]))
randIDs = []
while len(stackList) > 0:
    randID =  np.random.randint(len(stackList))
    randVal = stackList[randID]
    stackList.pop(randID)
    randIDs.append(randVal)   

In [34]:
len(master_index),len(randIDs),dfBG.shape

(453, 464, (464, 5))

In [27]:
master_index = dfBG.index.tolist()

#Initialize lists of layer indices, costs, LCP arrays, and LCP lines
sourceIds = []
pipeCosts = []
pipeArrays = []
pipeLines = []

#Copy the stack of cost distance arrays
arrStack_next = arrStack.copy()

arrLCP = arrPipes

#Loop through the remaining layers until all layers have been processed
for layerID in randIDs:
    #Get the biogas index
    bgIndex = master_index.pop(layerID)
    continue
    #Get the pipeCoords and pipeCost for the selected layer
    arrLayer = arrStack[layerID]
    #Get the layer containing the next cheapest source to the nearest pipe, and the location to connect
    pipeCoords, pipeCost = getPipeCoord(arrLayer,arrLCP,verbose=False)
    #Get the location of the farm associated with that layer
    farmCoords = getSourceCoordinates(arrStack,layerID,verbose=False)
    #Extract the LCP from the farm to the pipe coordinate
    arrLCP_next,coordList_next = getLCP(arrCosts,layerID,farmCoords,pipeCoords)
    #Construct the lcp line feature
    lineString = coordsToline(coordList_next,llx,lly,x_size,y_size)
    pipeLines.append(lineString)
    #Add the pipe cost and least cost path arrays to their respective list
    sourceIds.append(bgIndex)
    pipeCosts.append(pipeCost)
    pipeArrays.append(arrLCP_next * pipeCost) 
    #Update the pipe network array with the new LCP (max of binary arrays)
    arrLCP = np.maximum(arrLCP,arrLCP_next) 

IndexError: pop index out of range

In [ ]:
#Construct a feature class of the lines
import geopandas as gpd
#Contruct a geodataframe: start with the list of pipeline linestrings
gf = gpd.GeoDataFrame(pipeLines,columns=['geometry'])
#Add the biogas source IDs costs
gf['Ids'] = sourceIds
gf['Costs'] = pipeCosts
#Compute cumulative sum of costs
gf['SumCosts'] = gf.Costs.cumsum()
#Add the order added as an explicit column
gf['Order'] = gf.index + 1
#Add the projection
gf.crs = {'proj': 'eqdc', 'lat_0': 40, 'lon_0': -96, 'lat_1': 20, 'lat_2': 60, 
              'x_0': 0, 'y_0': 0, 'datum': 'NAD83', 'units': 'm', 'no_defs': True}


In [ ]:
#Join the yield values from the biogas dataframe
dfBG['Link'] = dfBG.index #Add the index as a link for joining
gf2 = pd.merge(gf,dfBG[['Link','Total Potential Methane Yield (scf/h)']],
               how='left',left_on='Ids',right_on='Link').drop('Link',axis=1)
#Compute cumulative yield
gf2['SumYield'] = gf2['Total Potential Methane Yield (scf/h)'].cumsum()

In [ ]:
#Save to a file
LCP_filename = '../scratch/LCPNetwork_Random.shp'
gf2.to_file(LCP_filename)